In [10]:
# import sklearn and random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import pandas as pd

In [16]:
# pull csv into df
origin_df = pd.read_csv("./Data_Extraction_Transform/dropped_cols_month_earthquake.csv", index_col= 0)
# covert time column
origin_df['time'] = pd.to_datetime(origin_df["time"])
# origin_df['time'] = pd.Series([val.time() for val in origin_df['time']])
origin_df.time = pd.to_datetime(origin_df.time).dt.strftime('%H:%M:%S')
origin_df.head()

,id,tsunami,felt,time,latitude,longitude,depth,mag,magType,gap,net,updated,place,type,locationSource
0,nn00818986,0,NaN,02:53:29,38.541200,-119.463900,3.60,1.30,ml,58.29,nn,2021-08-19T02:56:56.695Z,"3 km NNE of Walker, California",earthquake,nn
1,ci40014776,0,NaN,02:49:35,35.896667,-117.729167,4.48,0.72,ml,72.00,ci,2021-08-19T02:53:05.273Z,"17km ESE of Little Lake, CA",earthquake,ci
2,ak021am2mkg5,0,NaN,02:49:14,59.865100,-153.176400,102.60,1.60,ml,NaN,ak,2021-08-19T03:02:14.791Z,"52 km E of Pedro Bay, Alaska",earthquake,ak
3,nc73611791,0,NaN,02:46:16,38.784668,-122.751000,0.33,1.03,md,84.00,nc,2021-08-19T02:47:54.191Z,"1km NNE of The Geysers, CA",earthquake,nc
4,ak021am2l4yc,0,NaN,02:42:46,56.657400,-157.265100,0.50,2.90,ml,NaN,ak,2021-08-19T02:50:26.938Z,"80 km ENE of Chignik, Alaska",earthquake,ak


In [17]:
# grabbing our predictor and predicted columns from original df.
predictors = ["time", "latitude", "longitude"]
pred_values = ["mag", "depth"]
X = origin_df[predictors]
y = origin_df[pred_values]

In [18]:
# setting training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
best_model = None
best_score = 0

# I've tried running this model until the error rate stops improving, but it takes forever, so I'll revert it back to a finite loop first.
epsilon = 0.01
old_error = float('inf')
i = 1
while True:
    estimators = i*4
    model = RandomForestRegressor(n_estimators = estimators, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)
    print("n_estimators={}:".format(estimators))
    print("\tMean Squared Error:", mse, end = ", ")
    print("\tR2 score:", r2, end = ", ")
    print("\tModel score:", score)
    if abs(old_error - mse) <= epsilon:
        print(old_error)
        print(mse)
        print(old_error - mse)
        print("No improvement")
        break
    old_error = mse
    if best_score < score:
        best_score = score
        best_model = model
    i += 1

ValueError: could not convert string to float: '14:48:52'

In [20]:
# tsunami test
predictors = ["mag", "depth", "latitude","longitude"]
pred_values = ["tsunami"]
X = origin_df[predictors]
y = origin_df[pred_values]

In [21]:
# setting training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
best_model = None
best_score = 0

# I've tried running this model until the error rate stops improving, but it takes forever, so I'll revert it back to a finite loop first.
epsilon = 0.01
old_error = float('inf')
i = 1
while True:
    estimators = i*4
    model = RandomForestRegressor(n_estimators = estimators, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)
    print("n_estimators={}:".format(estimators))
    print("\tMean Squared Error:", mse, end = ", ")
    print("\tR2 score:", r2, end = ", ")
    print("\tModel score:", score)
    if abs(old_error - mse) <= epsilon:
        print(old_error)
        print(mse)
        print(old_error - mse)
        print("No improvement")
        break
    old_error = mse
    if best_score < score:
        best_score = score
        best_model = model
    i += 1

C:\Users\huayu\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()
C:\Users\huayu\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


n_estimators=4:
	Mean Squared Error: 0.00016416510318949343, 	R2 score: 0.7810857116034547, 	Model score: 0.7810857116034547
n_estimators=8:
	Mean Squared Error: 0.00011139774859287054, 	R2 score: 0.8514510185880586, 	Model score: 0.8514510185880586
0.00016416510318949343
0.00011139774859287054
5.276735459662289e-05
No improvement


In [23]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# max_err = max_error(y_pred, y_test)
r2 = r2_score(y_test, y_pred)
score = best_model.score(X_test, y_test)
print("Mean Squared Error:", mse)
print("R2 score:", r2)
print("Model score:", score)

Mean Squared Error: 0.00016416510318949343
R2 score: 0.7810857116034547
Model score: 0.7810857116034547
